# **Tools**
Models can request to call tools that perform tasks such as fetching data from a database or web scraping. Tools are pairings of:
1. A schema including the name of the tool, description, and parameters.
2. A function that implements the tool.

In [2]:
import os
# from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
# model = init_chat_model(model="qwen/qwen3-32b", temperature=0.9)
model = ChatGroq(model="qwen/qwen3-32b", temperature=0.9)

response = model.invoke("Write me an essay on DSA")
print(response.content)
# responses = model.stream("Write me an essay on DSA")

# for response in responses:
#     print(response.content)

<think>
Okay, the user wants an essay on DSA, which I know stands for Data Structures and Algorithms. Let me start by outlining the key points: definition, importance, types of data structures, algorithms, their applications, and maybe challenges in learning DSA.

First, I need to define DSA clearly. Maybe start with why they're essential in computer science. Then explain data structures like arrays, linked lists, stacks, queues, trees, graphs. For algorithms, cover searching, sorting, and optimization techniques like dynamic programming.

I should mention real-world applications to show relevance. Like how Google Maps uses graph algorithms or how databases use trees. Also, the role in problem-solving and efficiency. Maybe touch on time and space complexity analysis.

Challenges in learning DSA—maybe the initial difficulty, practice needed, and how to overcome it. Conclude by emphasizing their importance in tech careers.

Wait, the user might be a student needing a basic essay for a cl

## **Tools**

In [1]:
from langchain_core.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the current weather for a given location."""
    # This is a mock implementation for demonstration purposes.
    # In a real-world scenario, you would call a weather API here.
    return f"The weather in {location} is currently sunny with a temperature of 25°C."

In [18]:
tools = [get_weather]
model_with_tools = model.bind_tools(tools) #binding tools with LLM

In [ ]:
#Simple Case: One-Shot Tool Call
response = model_with_tools.invoke("What is the weather in Bengaluru?")

print(response)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Bengaluru. I need to use the get_weather function. The function requires a location parameter. Bengaluru is a city in India, so I should pass that as the location. Let me make sure the spelling is correct. Yes, "Bengaluru" is the correct spelling. I\'ll call the function with {"location": "Bengaluru"}.\n', 'tool_calls': [{'id': '65xkyp4se', 'function': {'arguments': '{"location":"Bengaluru"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 156, 'total_tokens': 267, 'completion_time': 0.188946869, 'completion_tokens_details': {'reasoning_tokens': 84}, 'prompt_time': 0.006259272, 'prompt_tokens_details': None, 'queue_time': 0.051282988, 'total_time': 0.195206141}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_

In [8]:
response.tool_calls #to check which tool was called

[{'name': 'get_weather',
  'args': {'location': 'Bengaluru'},
  'id': '65xkyp4se',
  'type': 'tool_call'}]

## **Tool Execution Loop**

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

query = "What is the weather in Kotdwar?"

#Step 1: Model decides tool calls
messages = [
    SystemMessage(
        content="You are a helpful, witty, and highly intelligent AI assistant."
    ),
    HumanMessage(content=query),
]

ai_response = model_with_tools.invoke(messages)

messages.append(ai_response)

In [ ]:
#Step 2: Execute Tools and Collect Results
for tool_call in ai_response.tool_calls:
    #Execute the tool with generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

#Step 3: Pass results back to the model for final response
final_response = model_with_tools.invoke(messages)

print(final_response.text)

The weather in Kotdwar is currently sunny and pleasant at 25°C. A great day to enjoy outdoor activities! 🌞


In [17]:
messages

[SystemMessage(content='You are a helpful, witty, and highly intelligent AI assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the weather in Kotdwar?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Kotdwar. Let me check what tools I have available. There\'s a function called get_weather that takes a location parameter. Since Kotdwar is a specific location, I should call that function with "Kotdwar" as the argument. I need to make sure the parameters are correctly formatted in JSON. Let me structure the tool call properly.\n', 'tool_calls': [{'id': '4z97gcvc9', 'function': {'arguments': '{"location":"Kotdwar"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 170, 'total_tokens': 279, 'completion_time': 0.164183418, 'completion_tokens_details': {'reasoning_tokens': 82

In [21]:
model_with_tools

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000020D9F0B2A50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020D9F0B3770>, model_name='qwen/qwen3-32b', temperature=0.9, model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the current weather for a given location.', 'parameters': {'properties': {'location': {'type': 'string'}}, 'required': ['location'], 'type': 'object'}}}]}, config={}, config_factories=[])